In [6]:
import FinanceDataReader as fdr
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm

BaseUrl = 'http://finance.naver.com/sise/entryJongmok.nhn?&page='

# 종목 코드 추출
stock_codes = []

for i in range(1, 21):  # range() 함수의 끝 파라미터는 포함되지 않는다.
    url = BaseUrl + str(i)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    items = soup.find_all('td', {'class': 'ctg'})
    for item in items:
        txt = item.a.get('href')  # 예: https://finance.naver.com/item/main.nhn?code=006390
        k = re.search('[\d]+', txt)  # 정규표현식 사용. [\d] 숫자표현, + : 반복
        if k:
            code = k.group()
            stock_codes.append(code)

print(stock_codes)

['005930', '000660', '373220', '207940', '005380', '012450', '068270', '000270', '329180', '105560', '035420', '055550', '012330', '042660', '138040', '005490', '028260', '009540', '034020', '086790', '259960', '000810', '032830', '035720', '010130', '015760', '011200', '051910', '096770', '033780', '316140', '030200', '010140', '064350', '024110', '006400', '402340', '066570', '017670', '352820', '267260', '323410', '003550', '018260', '003670', '034730', '000100', '009150', '047050', '086280', '326030', '047810', '003490', '272210', '042700', '003230', '090430', '079550', '006800', '010620', '005830', '021240', '267250', '010120', '010950', '180640', '051900', '032640', '009830', '161390', '005940', '000150', '271560', '029780', '241560', '016360', '000720', '071050', '298040', '377300', '034220', '006260', '450080', '011790', '251270', '028050', '022100', '001040', '000880', '097950', '078930', '036460', '035250', '128940', '039490', '175330', '011070', '138930', '004020', '454910',

In [7]:
len(stock_codes)

200

In [9]:
# 최종 수집할 데이터프레임
final_data = pd.DataFrame()

# 실패한 티커 기록
failed_tickers = []

# tqdm을 사용한 다운로드 진행 표시
for ticker in tqdm(stock_codes, desc="Downloading"):
    try:
        df = fdr.DataReader(ticker, start='2020-01-01', end='2024-12-31')[['Close']]
        df.rename(columns={'Close': str(ticker)}, inplace=True)
        final_data = pd.concat([final_data, df[str(ticker)]], axis=1)
    except Exception as e:
        failed_tickers.append(ticker)

# 결측치 선형보간
final_data = final_data.interpolate(method='linear')

# 파일로 저장
final_data.to_csv('./csv_files/kospi200_close.csv')

# 실패한 티커 저장
if failed_tickers:
    pd.Series(failed_tickers).to_csv('failed_tickers.csv', index=False)
    print(f"Some tickers failed: {failed_tickers}")
else:
    print("All tickers downloaded successfully!")


Downloading: 100%|██████████| 200/200 [00:41<00:00,  4.86it/s]


All tickers downloaded successfully!


In [11]:
stocks = pd.read_csv('csv_files/kospi200_close.csv')
stocks

,Unnamed: 0,005930,000660,373220,207940,005380,012450,068270,000270,329180,...,003620,005420,006650,002710,456040,271940,178920,093370,105630,006110
0,2020-01-02 00:00:00,55200,94700,NaN,423485,118000,35896,159612,42500,NaN,...,9867,5497,116000,2624,NaN,NaN,34800,7940,17350,4320
1,2020-01-03 00:00:00,55500,94500,NaN,417061,116000,36051,157395,42000,NaN,...,9748,5386,114000,2639,NaN,NaN,35200,8018,17250,4275
2,2020-01-06 00:00:00,55500,94300,NaN,407178,116000,36362,153404,41900,NaN,...,9511,5636,109000,2610,NaN,NaN,34000,7783,16950,4265
3,2020-01-07 00:00:00,55800,94000,NaN,414096,115500,35844,157838,42050,NaN,...,9511,5549,107500,2639,NaN,NaN,34800,7891,17300,4270
4,2020-01-08 00:00:00,56800,97400,NaN,403719,112000,35275,153404,41300,NaN,...,8966,5222,103000,2517,NaN,NaN,33000,7510,16400,4200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,2024-12-23 00:00:00,53500,169600,359000.0,949000,215000,326000,185192,100900,269500.0,...,3920,16020,79600,29150,62500.0,17620.0,17580,5110,14480,35400
1227,2024-12-24 00:00:00,54400,168500,350000.0,938000,216500,324000,185669,100600,272000.0,...,3875,15720,79100,28600,62000.0,16900.0,17560,5000,14350,35000
1228,2024-12-26 00:00:00,53600,170100,344500.0,937000,217500,323500,182900,102900,298000.0,...,3815,15500,78000,27650,61300.0,16450.0,17290,4945,14300,34200
1229,2024-12-27 00:00:00,53700,174500,348000.0,934000,215000,319000,180400,100900,289500.0,...,3785,15140,78000,27150,58900.0,16400.0,17000,4870,14050,33400


In [12]:
# NaN 값이 하나라도 있는 열만 선택
nan_columns = stocks.columns[stocks.isnull().any()]

# 그 열들에 대해 NaN 개수 출력
nan_counts = stocks[nan_columns].isnull().sum()

# NaN이 존재하는 열의 개수
nan_columns_count = len(nan_columns)

print(nan_counts)
print(f"NaN이 있는 열의 개수: {nan_columns_count}")


373220     514
329180     426
259960     399
402340     472
352820     194
323410     397
326030     124
377300     454
450080     958
454910     928
302440     298
489790    1168
278470    1025
383220     342
375500     263
361610     335
457190     843
137310     382
456040     842
271940     414
dtype: int64
NaN이 있는 열의 개수: 20


In [13]:
# NaN 있는 열 리스트 만들기
nan_columns_list = stocks.columns[stocks.isnull().any()].tolist()

# NaN 있는 열을 드랍(drop)해서 새로운 DataFrame 만들기
stocks_no_nan = stocks.drop(columns=nan_columns_list)

# 결과 확인
print(stocks_no_nan.shape)
stocks_no_nan

(1231, 181)


,Unnamed: 0,005930,000660,207940,005380,012450,068270,000270,105560,035420,...,069260,001430,003620,005420,006650,002710,178920,093370,105630,006110
0,2020-01-02 00:00:00,55200,94700,423485,118000,35896,159612,42500,46550,182500,...,20600,15000,9867,5497,116000,2624,34800,7940,17350,4320
1,2020-01-03 00:00:00,55500,94500,417061,116000,36051,157395,42000,47150,181500,...,20450,14900,9748,5386,114000,2639,35200,8018,17250,4275
2,2020-01-06 00:00:00,55500,94300,407178,116000,36362,153404,41900,46600,180500,...,20000,14550,9511,5636,109000,2610,34000,7783,16950,4265
3,2020-01-07 00:00:00,55800,94000,414096,115500,35844,157838,42050,47000,187000,...,20050,14450,9511,5549,107500,2639,34800,7891,17300,4270
4,2020-01-08 00:00:00,56800,97400,403719,112000,35275,153404,41300,46150,183000,...,19300,13800,8966,5222,103000,2517,33000,7510,16400,4200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,2024-12-23 00:00:00,53500,169600,949000,215000,326000,185192,100900,87000,201000,...,18570,20450,3920,16020,79600,29150,17580,5110,14480,35400
1227,2024-12-24 00:00:00,54400,168500,938000,216500,324000,185669,100600,86500,202500,...,18540,20300,3875,15720,79100,28600,17560,5000,14350,35000
1228,2024-12-26 00:00:00,53600,170100,937000,217500,323500,182900,102900,85500,199900,...,18440,20250,3815,15500,78000,27650,17290,4945,14300,34200
1229,2024-12-27 00:00:00,53700,174500,934000,215000,319000,180400,100900,85000,197700,...,17570,19850,3785,15140,78000,27150,17000,4870,14050,33400


In [14]:
# 첫 번째 열 이름을 'Date'로 변경
stocks_no_nan = stocks_no_nan.rename(columns={stocks_no_nan.columns[0]: 'Date'})

# 'Date' 열을 datetime 형식으로 변환
stocks_no_nan['Date'] = pd.to_datetime(stocks_no_nan['Date'])

# 날짜를 yyyy-mm-dd 형식으로 포맷 (필요하면)
stocks_no_nan['Date'] = stocks_no_nan['Date'].dt.strftime('%Y-%m-%d')

# 결과 확인
stocks_no_nan


,Date,005930,000660,207940,005380,012450,068270,000270,105560,035420,...,069260,001430,003620,005420,006650,002710,178920,093370,105630,006110
0,2020-01-02,55200,94700,423485,118000,35896,159612,42500,46550,182500,...,20600,15000,9867,5497,116000,2624,34800,7940,17350,4320
1,2020-01-03,55500,94500,417061,116000,36051,157395,42000,47150,181500,...,20450,14900,9748,5386,114000,2639,35200,8018,17250,4275
2,2020-01-06,55500,94300,407178,116000,36362,153404,41900,46600,180500,...,20000,14550,9511,5636,109000,2610,34000,7783,16950,4265
3,2020-01-07,55800,94000,414096,115500,35844,157838,42050,47000,187000,...,20050,14450,9511,5549,107500,2639,34800,7891,17300,4270
4,2020-01-08,56800,97400,403719,112000,35275,153404,41300,46150,183000,...,19300,13800,8966,5222,103000,2517,33000,7510,16400,4200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,2024-12-23,53500,169600,949000,215000,326000,185192,100900,87000,201000,...,18570,20450,3920,16020,79600,29150,17580,5110,14480,35400
1227,2024-12-24,54400,168500,938000,216500,324000,185669,100600,86500,202500,...,18540,20300,3875,15720,79100,28600,17560,5000,14350,35000
1228,2024-12-26,53600,170100,937000,217500,323500,182900,102900,85500,199900,...,18440,20250,3815,15500,78000,27650,17290,4945,14300,34200
1229,2024-12-27,53700,174500,934000,215000,319000,180400,100900,85000,197700,...,17570,19850,3785,15140,78000,27150,17000,4870,14050,33400


In [16]:
stocks_no_nan.to_csv('./csv_files/kospi200_no_nan.csv', index = False)

In [17]:
nan_columns_list

['373220',
 '329180',
 '259960',
 '402340',
 '352820',
 '323410',
 '326030',
 '377300',
 '450080',
 '454910',
 '302440',
 '489790',
 '278470',
 '383220',
 '375500',
 '361610',
 '457190',
 '137310',
 '456040',
 '271940']

In [18]:
import FinanceDataReader as fdr

# 네가 준 티커 리스트
target_tickers = nan_columns_list

# 전체 한국 상장 종목 불러오기
krx = fdr.StockListing('KRX')

# 티커-종목명 매핑
ticker_name_dict = {}

for ticker in target_tickers:
    name = krx.loc[krx['Code'] == ticker, 'Name']
    if not name.empty:
        ticker_name_dict[ticker] = name.values[0]
    else:
        ticker_name_dict[ticker] = None  # 못 찾은 경우 None 처리

# 결과 확인
print(ticker_name_dict)


{'373220': 'LG에너지솔루션', '329180': 'HD현대중공업', '259960': '크래프톤', '402340': 'SK스퀘어', '352820': '하이브', '323410': '카카오뱅크', '326030': 'SK바이오팜', '377300': '카카오페이', '450080': '에코프로머티', '454910': '두산로보틱스', '302440': 'SK바이오사이언스', '489790': '한화비전', '278470': '에이피알', '383220': 'F&F', '375500': 'DL이앤씨', '361610': 'SK아이이테크놀로지', '457190': '이수스페셜티케미컬', '137310': '에스디바이오센서', '456040': 'OCI', '271940': '일진하이솔루스'}


In [20]:
print(ticker_name_dict.values())

dict_values(['LG에너지솔루션', 'HD현대중공업', '크래프톤', 'SK스퀘어', '하이브', '카카오뱅크', 'SK바이오팜', '카카오페이', '에코프로머티', '두산로보틱스', 'SK바이오사이언스', '한화비전', '에이피알', 'F&F', 'DL이앤씨', 'SK아이이테크놀로지', '이수스페셜티케미컬', '에스디바이오센서', 'OCI', '일진하이솔루스'])


In [21]:
# 추출할 날짜 리스트
target_dates = ['2020-12-30', '2021-12-30', '2022-12-29', '2023-12-28', '2024-12-30']

# 'Date' 열이 있는지 확인 후, Date를 문자열로 맞춰줌
if not pd.api.types.is_string_dtype(stocks_no_nan['Date']):
    stocks_no_nan['Date'] = stocks_no_nan['Date'].astype(str)

# 해당 날짜에 해당하는 행만 추출
stocks_selected = stocks_no_nan[stocks_no_nan['Date'].isin(target_dates)].reset_index(drop=True)

# 결과 확인
stocks_selected

,Date,005930,000660,207940,005380,012450,068270,000270,105560,035420,...,069260,001430,003620,005420,006650,002710,178920,093370,105630,006110
0,2020-12-30,81000,118500,816333,192000,29482,324169,62400,43400,292500,...,25650,10600,13140,10497,230000,5290,37000,11524,17500,11550
1,2021-12-30,78300,131000,892432,209000,49655,182162,82200,55000,378500,...,23150,19900,13140,13854,183000,9680,55100,22560,21950,27000
2,2022-12-29,55300,75000,821000,151000,76137,153292,59300,48500,177500,...,20000,16300,13140,20220,171500,9350,28900,10450,15400,41400
3,2023-12-28,78500,141500,760000,203500,128793,192450,100000,54100,224000,...,21500,24850,8680,39000,153000,58800,30500,11700,21050,106400
4,2024-12-30,53200,173900,949000,212000,326500,187500,100700,82900,198900,...,17050,19760,3770,15300,77700,27200,16890,4975,14170,33850


In [23]:
# 예시로 nan_columns_list 가 있다고 가정
# nan_columns_list = ['356860', '464500', '212710', ...] 

# nan_columns_list에 있는 것들 제외하고 새로운 리스트 만들기
filtered_tickers = [ticker for ticker in stock_codes if ticker not in nan_columns_list]

print(filtered_tickers)


['005930', '000660', '207940', '005380', '012450', '068270', '000270', '105560', '035420', '055550', '012330', '042660', '138040', '005490', '028260', '009540', '034020', '086790', '000810', '032830', '035720', '010130', '015760', '011200', '051910', '096770', '033780', '316140', '030200', '010140', '064350', '024110', '006400', '066570', '017670', '267260', '003550', '018260', '003670', '034730', '000100', '009150', '047050', '086280', '047810', '003490', '272210', '042700', '003230', '090430', '079550', '006800', '010620', '005830', '021240', '267250', '010120', '010950', '180640', '051900', '032640', '009830', '161390', '005940', '000150', '271560', '029780', '241560', '016360', '000720', '071050', '298040', '034220', '006260', '011790', '251270', '028050', '022100', '001040', '000880', '097950', '078930', '036460', '035250', '128940', '039490', '175330', '011070', '138930', '004020', '036570', '011780', '017800', '052690', '026960', '011170', '139480', '004370', '018880', '012750',

In [24]:
# 네가 준 티커 리스트
target_tickers = filtered_tickers

# 전체 한국 상장 종목 불러오기
krx = fdr.StockListing('KRX')

# 티커-종목명 매핑
ticker_filtered_name_dict = {}

for ticker in target_tickers:
    name = krx.loc[krx['Code'] == ticker, 'Name']
    if not name.empty:
        ticker_filtered_name_dict[ticker] = name.values[0]
    else:
        ticker_filtered_name_dict[ticker] = None  # 못 찾은 경우 None 처리

# 결과 확인
print(ticker_filtered_name_dict)


{'005930': '삼성전자', '000660': 'SK하이닉스', '207940': '삼성바이오로직스', '005380': '현대차', '012450': '한화에어로스페이스', '068270': '셀트리온', '000270': '기아', '105560': 'KB금융', '035420': 'NAVER', '055550': '신한지주', '012330': '현대모비스', '042660': '한화오션', '138040': '메리츠금융지주', '005490': 'POSCO홀딩스', '028260': '삼성물산', '009540': 'HD한국조선해양', '034020': '두산에너빌리티', '086790': '하나금융지주', '000810': '삼성화재', '032830': '삼성생명', '035720': '카카오', '010130': '고려아연', '015760': '한국전력', '011200': 'HMM', '051910': 'LG화학', '096770': 'SK이노베이션', '033780': 'KT&G', '316140': '우리금융지주', '030200': 'KT', '010140': '삼성중공업', '064350': '현대로템', '024110': '기업은행', '006400': '삼성SDI', '066570': 'LG전자', '017670': 'SK텔레콤', '267260': 'HD현대일렉트릭', '003550': 'LG', '018260': '삼성에스디에스', '003670': '포스코퓨처엠', '034730': 'SK', '000100': '유한양행', '009150': '삼성전기', '047050': '포스코인터내셔널', '086280': '현대글로비스', '047810': '한국항공우주', '003490': '대한항공', '272210': '한화시스템', '042700': '한미반도체', '003230': '삼양식품', '090430': '아모레퍼시픽', '079550': 'LIG넥스원', '006800': '미래에셋증권', '010620': 'H

In [25]:
# 'Date' 열을 제외한 나머지 열 이름만 바꿔야 하므로
columns_renamed = {}

for col in stocks_selected.columns:
    if col == 'Date':
        columns_renamed[col] = 'Date'
    else:
        columns_renamed[col] = ticker_filtered_name_dict.get(col, col)  # 매칭 안 되면 기존 col 이름 유지

# 컬럼명 변경
stocks_selected.rename(columns=columns_renamed, inplace=True)

# 결과 확인
stocks_selected.head()


,Date,삼성전자,SK하이닉스,삼성바이오로직스,현대차,한화에어로스페이스,셀트리온,기아,KB금융,NAVER,...,TKG휴켐스,세아베스틸지주,KG모빌리티,코스모화학,대한유화,TCC스틸,PI첨단소재,후성,한세실업,삼아알미늄
0,2020-12-30,81000,118500,816333,192000,29482,324169,62400,43400,292500,...,25650,10600,13140,10497,230000,5290,37000,11524,17500,11550
1,2021-12-30,78300,131000,892432,209000,49655,182162,82200,55000,378500,...,23150,19900,13140,13854,183000,9680,55100,22560,21950,27000
2,2022-12-29,55300,75000,821000,151000,76137,153292,59300,48500,177500,...,20000,16300,13140,20220,171500,9350,28900,10450,15400,41400
3,2023-12-28,78500,141500,760000,203500,128793,192450,100000,54100,224000,...,21500,24850,8680,39000,153000,58800,30500,11700,21050,106400
4,2024-12-30,53200,173900,949000,212000,326500,187500,100700,82900,198900,...,17050,19760,3770,15300,77700,27200,16890,4975,14170,33850


In [26]:
stocks_selected

,Date,삼성전자,SK하이닉스,삼성바이오로직스,현대차,한화에어로스페이스,셀트리온,기아,KB금융,NAVER,...,TKG휴켐스,세아베스틸지주,KG모빌리티,코스모화학,대한유화,TCC스틸,PI첨단소재,후성,한세실업,삼아알미늄
0,2020-12-30,81000,118500,816333,192000,29482,324169,62400,43400,292500,...,25650,10600,13140,10497,230000,5290,37000,11524,17500,11550
1,2021-12-30,78300,131000,892432,209000,49655,182162,82200,55000,378500,...,23150,19900,13140,13854,183000,9680,55100,22560,21950,27000
2,2022-12-29,55300,75000,821000,151000,76137,153292,59300,48500,177500,...,20000,16300,13140,20220,171500,9350,28900,10450,15400,41400
3,2023-12-28,78500,141500,760000,203500,128793,192450,100000,54100,224000,...,21500,24850,8680,39000,153000,58800,30500,11700,21050,106400
4,2024-12-30,53200,173900,949000,212000,326500,187500,100700,82900,198900,...,17050,19760,3770,15300,77700,27200,16890,4975,14170,33850


In [27]:
# 'Date' 열 제외한 나머지 열을 float로 변환
cols_to_convert = [col for col in stocks_selected.columns if col != 'Date']
stocks_selected[cols_to_convert] = stocks_selected[cols_to_convert].astype(float)

# 결과 확인
print(stocks_selected.dtypes)


Date         object
삼성전자        float64
SK하이닉스      float64
삼성바이오로직스    float64
현대차         float64
             ...   
TCC스틸       float64
PI첨단소재      float64
후성          float64
한세실업        float64
삼아알미늄       float64
Length: 181, dtype: object


In [28]:
stocks_selected

,Date,삼성전자,SK하이닉스,삼성바이오로직스,현대차,한화에어로스페이스,셀트리온,기아,KB금융,NAVER,...,TKG휴켐스,세아베스틸지주,KG모빌리티,코스모화학,대한유화,TCC스틸,PI첨단소재,후성,한세실업,삼아알미늄
0,2020-12-30,81000.0,118500.0,816333.0,192000.0,29482.0,324169.0,62400.0,43400.0,292500.0,...,25650.0,10600.0,13140.0,10497.0,230000.0,5290.0,37000.0,11524.0,17500.0,11550.0
1,2021-12-30,78300.0,131000.0,892432.0,209000.0,49655.0,182162.0,82200.0,55000.0,378500.0,...,23150.0,19900.0,13140.0,13854.0,183000.0,9680.0,55100.0,22560.0,21950.0,27000.0
2,2022-12-29,55300.0,75000.0,821000.0,151000.0,76137.0,153292.0,59300.0,48500.0,177500.0,...,20000.0,16300.0,13140.0,20220.0,171500.0,9350.0,28900.0,10450.0,15400.0,41400.0
3,2023-12-28,78500.0,141500.0,760000.0,203500.0,128793.0,192450.0,100000.0,54100.0,224000.0,...,21500.0,24850.0,8680.0,39000.0,153000.0,58800.0,30500.0,11700.0,21050.0,106400.0
4,2024-12-30,53200.0,173900.0,949000.0,212000.0,326500.0,187500.0,100700.0,82900.0,198900.0,...,17050.0,19760.0,3770.0,15300.0,77700.0,27200.0,16890.0,4975.0,14170.0,33850.0


In [29]:
stocks_selected.to_csv('./csv_files/kospi200_year_end.csv')